# Использование последовательного анализа

Ниже приведены *примеры использования* последовательного анализа для *бинарных величин* (стандартных тестов конверсий). Вы *можете* смело клонировать себе проект и перебирать различные параметры, чтобы оценить простоту использования.

Пара важных замечаний.
* Для наглядности используются не данные реальных тестов, а *синтетические данные* из генератора случайных чисел. Так мы имеем возможность наблюдать поведение последовательного анализа при *различных значениях конверсий*.
* *Длительность* последовательного теста *случайна*, поэтому при перезапуске кода длительность теста вообще говоря будет разной.
* Ради простоты использования классы критериев *реализованы одним файлом*. Вы можете копировать код `BinaryOneSampleSprt` или `BinaryTwoSampleSprt` в свой проект и импортировать в любом блокноте уже готовую реализацию.

# Импорты функций

In [1]:
from scipy.stats import bernoulli

from binary.sprt import BinaryOneSampleSprt, BinaryTwoSampleSprt

## Случай двух выборок

In [2]:
p0 = 0.07
d = 0.005
alpha = 0.05
beta = 0.2
alternative = "two-sided"

In [3]:
sprt = BinaryTwoSampleSprt(p0, d, alpha, beta, 
                           alternative=alternative)

decision_desc = "Тест продолжается"
p1 = p0
p2 = p0-d

while decision_desc == "Тест продолжается":
    sample_num = 1 + bernoulli.rvs(0.5)
    first_sample_flg = sample_num == 1
    
    if first_sample_flg:
        x = bernoulli.rvs(p1)
    else:
        x = bernoulli.rvs(p2)
        
    decision_desc = sprt.append(x, first_sample_flg)
    
    if decision_desc != "Тест продолжается":
        stop_flg = True

print(decision_desc)
print(f"Длительность теста для 1 выборки: {sprt.stop_first_sample_size}, "
      + f"для 2 выборки: {sprt.stop_second_sample_size}.")

Тест остановлен, справедлива альтернатива p1 > p2
Длительность теста для 1 выборки: 12798, для 2 выборки: 12618.


Тест закончился с *правильным решением*: справедлива альтернатива о том, 
что конверсия в первой выборке больше, чем во второй. 
При этом в каждой вариации потребовалось около *12 700 наблюдений*.

Отметич, что для классического теста с фиксированной длительностью на каждую вариацию
потребовалось бы [42 212 наблюдения](https://abntester.com/#/calculation/two-sample?p=7&mde=0.5&alpha=5&beta=20&leftProportion=50&type=BINARY&alternative=TWO_SIDED&showResult=1).

## Случай одной выборки

In [4]:
p0 = 0.07
d = 0.005
alpha = 0.05
beta = 0.2
alternative = "less"

In [5]:
sprt = BinaryOneSampleSprt(p0, d, alpha, beta, 
                           alternative=alternative)

decision_desc = "Тест продолжается"

while decision_desc == "Тест продолжается":
    x = bernoulli.rvs(p0)
    decision_desc = sprt.append(x)
    
    if decision_desc != "Тест продолжается":
        stop_flg = True

print(decision_desc)
print(f"Длительность теста: {sprt.stop_sample_size}")

Тест остановлен, справедлива гипотеза p >= p0
Длительность теста: 8695


Тест закончился с *правильным решением*: справедлива гипотеза о том, что конверсия не уменьшилась. При этом для теста потребовалось *8 695 наблюдений*.

Отметич, что для классического теста с фиксированной длительностью потребовалось бы [15 734 наблюдения](https://abntester.com/#/calculation/one-sample?p=7&mde=0.5&alpha=5&beta=20&type=BINARY&alternative=LEFT_SIDED&showResult=1).